In [121]:
import cv2
import math as m
import mediapipe as mp

In [122]:
# Find Distance between 2 points
def findDistance(x1, y1, x2, y2):
    dist = m.sqrt((x2-x1)**2+(y2-y1)**2)
    return dist

# Calculate angle
def findAngle(x1, y1, x2, y2):
    theta = m.acos((y2 -y1)*(-y1) / (m.sqrt((x2 - x1)**2 + (y2 - y1)**2) * y1))
    degree = int(180/m.pi)*theta
    return degree

# Calculate difference of x-coordinate of two points
def findX(x_knee,x_hand):
  X = x_hand - x_knee
  return X

In [123]:
# Font (For OpenCV Video)
font = cv2.FONT_HERSHEY_SIMPLEX

# Colors 
blue = (255, 127, 0)
red = (50, 50, 255)
green = (127, 255, 0)
dark_blue = (127, 20, 0)
light_green = (127, 233, 100)
yellow = (0, 255, 255)
pink = (255, 0, 255)

In [124]:
# Using Mediapipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Choose which video to use
# ((For webcam input replace file name with 0))
file_name = './videos/don/don9.mp4'
cap = cv2.VideoCapture(file_name)

# CV2  properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Output video details
video_output = cv2.VideoWriter('asldjald.mp4', fourcc, fps, frame_size)

In [125]:
print('Starting...')
steps = 0
stage = None
max_dis = 0
access = 1
ball_release = None
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
while cap.isOpened():
    # Capture frames
    success, image = cap.read() 
    if not success:
        print("No frames left to process!!!")
        break
    # Get fps, height and width
    fps = cap.get(cv2.CAP_PROP_FPS)
    h, w = image.shape[:2]
    # Convert the BGR image to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Process the frame with Mediapipe Pose
    keypoints = pose.process(image)
    # Convert the image back to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    #============ Getting landmarks ============
    lm = keypoints.pose_landmarks
    lmPose = mp_pose.PoseLandmark

    # Left shoulder
    l_shldr_x = int(lm.landmark[lmPose.LEFT_SHOULDER].x * w)
    l_shldr_y = int(lm.landmark[lmPose.LEFT_SHOULDER].y * h)
    # Right shoulder
    r_shldr_x = int(lm.landmark[lmPose.RIGHT_SHOULDER].x * w)
    r_shldr_y = int(lm.landmark[lmPose.RIGHT_SHOULDER].y * h)
    # Left ear
    l_ear_x = int(lm.landmark[lmPose.LEFT_EAR].x * w)
    l_ear_y = int(lm.landmark[lmPose.LEFT_EAR].y * h)
    # Left hip
    l_hip_x = int(lm.landmark[lmPose.LEFT_HIP].x * w)
    l_hip_y = int(lm.landmark[lmPose.LEFT_HIP].y * h)

    # Ankles for feet distance calculation
    l_ank_x = int(lm.landmark[lmPose.LEFT_ANKLE].x * 100)
    l_ank_y = int(lm.landmark[lmPose.LEFT_ANKLE].y * 100)
    r_ank_x = int(lm.landmark[lmPose.RIGHT_ANKLE].x * 100)
    r_ank_y = int(lm.landmark[lmPose.RIGHT_ANKLE].y * 100)

     #ankles for feet distance calculation
    # l_ank_x = int(lm.landmark[lmPose.LEFT_ANKLE].x * w)
    # l_ank_y = int(lm.landmark[lmPose.LEFT_ANKLE].y * h)
    # r_ank_x = int(lm.landmark[lmPose.RIGHT_ANKLE].x * w)
    # r_ank_y = int(lm.landmark[lmPose.RIGHT_ANKLE].y * h)

    r_wrist_x = int(lm.landmark[lmPose.RIGHT_WRIST].x * w)
    r_knee_x = int(lm.landmark[lmPose.RIGHT_KNEE].x * w)
    l_knee_x = int(lm.landmark[lmPose.LEFT_KNEE].x * w)

    #============ Functions ============
    # Check for Camera Alignment to be in Proper Sideview
    offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
    if offset < 100:
        cv2.putText(image, str(int(offset)) + ' Aligned', (w - 150, 30), font, 0.9, green, 2)
    else:
        cv2.putText(image, str(int(offset)) + ' Not Aligned', (w - 150, 30), font, 0.9, red, 2)

    feetDist = findDistance(l_ank_x, l_ank_y, r_ank_x, r_ank_y)

    # Steps Counter (To be improved - ie Thresholds improvements)
    if steps < 5:
        if feetDist > 10 and stage == 'up':
            steps += 1
            print("step ",steps)
            print("left ",l_ank_y)
            print("right ",r_ank_y)
            stage = "down"
        elif feetDist < 5:
            stage = "up"


    if steps == 5:
      if feetDist > max_dis and access == 1:
        max_dis = feetDist
      elif ball_release == None:
        print(feetDist)
        access = 0
        # check distance between hand and knee
        ball_train_feet_dis = findX(r_knee_x, r_wrist_x)
        ball_slide_feet_dis = findX(l_knee_x, r_wrist_x)
        if ball_train_feet_dis < 0:
          ball_release = "Late"
        elif ball_train_feet_dis > 0 and ball_slide_feet_dis < 0:
          ball_release = "Traditional"
        elif ball_slide_feet_dis > 0:
          ball_release = "Early"

    
    # Calculate torso and neck angles
    neck_inclination = findAngle(l_shldr_x, l_shldr_y, l_ear_x, l_ear_y)
    torso_inclination = findAngle(l_hip_x, l_hip_y, l_shldr_x, l_shldr_y)

    #============ Annotations onto video ============
    # # Draw landmarks
    # cv2.circle(image, (l_shldr_x, l_shldr_y), 7, yellow, -1)
    # cv2.circle(image, (l_ear_x, l_ear_y), 7, yellow, -1)
    # cv2.circle(image, (l_shldr_x, l_shldr_y - 100), 7, yellow, -1)
    # # Right shoulder is pink ball
    # cv2.circle(image, (r_shldr_x, r_shldr_y), 7, pink, -1)
    # cv2.circle(image, (l_hip_x, l_hip_y), 7, yellow, -1)
    # cv2.circle(image, (l_hip_x, l_hip_y - 100), 7, yellow, -1)
    mp_drawing.draw_landmarks(
        image,
        keypoints.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

    # Text for Neck/Torso Angle, Feet distance & Steps
    # angle_text_string = 'Neck : ' + str(int(neck_inclination)) + '  Torso : ' + str(int(torso_inclination)) + ' Feet distance: '+ str(int(feetDist)) + ' Steps: '+ str(int(steps))
    angle_text_string = 'Stage : ' + str(stage) +  ' Feet distance: '+ str(int(feetDist)) + ' Steps: '+ str(int(steps)) + 'Release: '+ str(ball_release)
    cv2.putText(image, angle_text_string, (10, 50), font, 0.8, dark_blue, 4)

    # # Display angles on the annotation
    # cv2.putText(image, str(int(neck_inclination)), (l_shldr_x + 10, l_shldr_y), font, 1, pink, 2)
    # cv2.putText(image, str(int(torso_inclination)), (l_hip_x + 10, l_hip_y), font, 1, pink, 2)

    # # Join landmarks
    # cv2.line(image, (l_shldr_x, l_shldr_y), (l_ear_x, l_ear_y), green, 4)
    # cv2.line(image, (l_shldr_x, l_shldr_y), (l_shldr_x, l_shldr_y - 100), green, 4)
    # cv2.line(image, (l_hip_x, l_hip_y), (l_shldr_x, l_shldr_y), green, 4)
    # cv2.line(image, (l_hip_x, l_hip_y), (l_hip_x, l_hip_y - 100), green, 4)
    

    # Write frames.
    video_output.write(image)
print('Video is done!')
cap.release()
video_output.release()

Starting...
step  1
left  81
right  81
step  2
left  80
right  82
step  3
left  78
right  81
step  4
left  79
right  78
No frames left to process!!!
Video is done!
